# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments  \
0  Sep  2 2022 11:47AM  245632        12     Cart120101   
1  Sep  2 2022 11:43AM  245630        20     ALU0007808   
2  Sep  2 2022 11:43AM  245630        20  ALUR038387200   
3  Sep  2 2022 11:43AM  245630        20  ALUR084516331   
4  Sep  2 2022 11:43AM  245630        20  ALUR237702375   
5  Sep  2 2022 11:43AM  245630        20  ALUR622914030   
6  Sep  2 2022 11:43AM  245630        20  ALUR776953997   
7  Sep  2 2022 11:43AM  245631        10      MSP213658   
8  Sep  2 2022 11:29AM  245627        16        8467389   
9  Sep  2 2022 11:29AM  245627        16        8468535   

                         Customer ShipmentStatus  
0                       Hush Hush       Released  
1               Alumier Labs Inc.       Released  
2               Alumier Labs Inc.       Released  
3               Alumier Labs Inc.       Released  
4               Alumier Labs Inc.       Released  
5               Alumier Labs Inc.       Released  
6               Alumier Labs Inc.       Released  
7                Methapharm, Inc.       Released  
8  Sartorius Bioprocess Solutions       Released  
9  Sartorius Bioprocess Solutions       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
33  245626       Released          1
34  245627       Released          2
35  245630       Released          6
36  245631       Released          1
37  245632       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
245626                NaN        NaN       1.0
245627                NaN        NaN       2.0
245630                NaN        NaN       6.0
245631                NaN        NaN       1.0
245632                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245455                5.0        1.0       0.0
245482                0.0       12.0      19.0
245562                5.0        8.0       1.0
245563                0.0        0.0       1.0
245564                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245455                  5          1         0
245482                  0         12        19
245562                  5          8         1
245563                  0          0         1
245564                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               245455          5          1         0
1               245482          0         12        19
2               245562          5          8         1
3               245563          0          0         1
4               245564          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               245455         5         1         
1               245482                  12       19
2               245562         5         8        1
3               245563                            1
4               245564                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Sep  2 2022 11:47AM  245632        12                       Hush Hush
1   Sep  2 2022 11:43AM  245630        20               Alumier Labs Inc.
7   Sep  2 2022 11:43AM  245631        10                Methapharm, Inc.
8   Sep  2 2022 11:29AM  245627        16  Sartorius Bioprocess Solutions
10  Sep  2 2022 11:20AM  245626        16   Sartorius Stedim Filters Inc.
11  Sep  2 2022 11:20AM  245625        16   Sartorius Stedim Filters Inc.
12  Sep  2 2022 11:17AM  245624        18                   TASA USA Inc.
13  Sep  2 2022 11:14AM  245622        20        ACI Healthcare USA, Inc.
24  Sep  2 2022 10:36AM  245619        10                Methapharm, Inc.
46  Sep  2 2022 10:24AM  245482        15            Person & Covey, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Sep  2 2022 11:47AM  245632        12                       Hush Hush   
1  Sep  2 2022 11:43AM  245630        20               Alumier Labs Inc.   
2  Sep  2 2022 11:43AM  245631        10                Methapharm, Inc.   
3  Sep  2 2022 11:29AM  245627        16  Sartorius Bioprocess Solutions   
4  Sep  2 2022 11:20AM  245626        16   Sartorius Stedim Filters Inc.   
5  Sep  2 2022 11:20AM  245625        16   Sartorius Stedim Filters Inc.   
6  Sep  2 2022 11:17AM  245624        18                   TASA USA Inc.   
7  Sep  2 2022 11:14AM  245622        20        ACI Healthcare USA, Inc.   
8  Sep  2 2022 10:36AM  245619        10                Methapharm, Inc.   
9  Sep  2 2022 10:24AM  245482        15            Person & Covey, Inc.   

  Completed Executing Released  
0                            1  
1                            6  
2                            1  
3                            2  
4                            1  
5                            1  
6                            1  
7                           11  
8                           22  
9                  12       19

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Sep  2 2022 11:47AM  245632        12                       Hush Hush   
1  Sep  2 2022 11:43AM  245630        20               Alumier Labs Inc.   
2  Sep  2 2022 11:43AM  245631        10                Methapharm, Inc.   
3  Sep  2 2022 11:29AM  245627        16  Sartorius Bioprocess Solutions   
4  Sep  2 2022 11:20AM  245626        16   Sartorius Stedim Filters Inc.   
5  Sep  2 2022 11:20AM  245625        16   Sartorius Stedim Filters Inc.   
6  Sep  2 2022 11:17AM  245624        18                   TASA USA Inc.   
7  Sep  2 2022 11:14AM  245622        20        ACI Healthcare USA, Inc.   
8  Sep  2 2022 10:36AM  245619        10                Methapharm, Inc.   
9  Sep  2 2022 10:24AM  245482        15            Person & Covey, Inc.   

  Released Executing Completed  
0        1                      
1        6                      
2        1                      
3        2                      
4        1                      
5        1                      
6        1                      
7       11                      
8       22                      
9       19        12

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Sep  2 2022 11:47AM  245632        12                       Hush Hush   
1  Sep  2 2022 11:43AM  245630        20               Alumier Labs Inc.   
2  Sep  2 2022 11:43AM  245631        10                Methapharm, Inc.   
3  Sep  2 2022 11:29AM  245627        16  Sartorius Bioprocess Solutions   
4  Sep  2 2022 11:20AM  245626        16   Sartorius Stedim Filters Inc.   

  Released Executing Completed  
0        1                      
1        6                      
2        1                      
3        2                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Sep  2 2022 11:47AM  245632        12                       Hush Hush   
1  Sep  2 2022 11:43AM  245630        20               Alumier Labs Inc.   
2  Sep  2 2022 11:43AM  245631        10                Methapharm, Inc.   
3  Sep  2 2022 11:29AM  245627        16  Sartorius Bioprocess Solutions   
4  Sep  2 2022 11:20AM  245626        16   Sartorius Stedim Filters Inc.   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       6.0        NaN        NaN  
2       1.0        NaN        NaN  
3       2.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Sep  2 2022 11:47AM  245632        12                       Hush Hush   
1  Sep  2 2022 11:43AM  245630        20               Alumier Labs Inc.   
2  Sep  2 2022 11:43AM  245631        10                Methapharm, Inc.   
3  Sep  2 2022 11:29AM  245627        16  Sartorius Bioprocess Solutions   
4  Sep  2 2022 11:20AM  245626        16   Sartorius Stedim Filters Inc.   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       6.0        0.0        0.0  
2       1.0        0.0        0.0  
3       2.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1964855      80.0        0.0        0.0
12          491211       2.0        0.0        0.0
15          736671      31.0       18.0        0.0
16         2456016      10.0        1.0        0.0
17          245585       1.0        0.0        0.0
18          245624       1.0        0.0        0.0
20         1964545      22.0        9.0       10.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1964855      80.0        0.0        0.0
1        12   491211       2.0        0.0        0.0
2        15   736671      31.0       18.0        0.0
3        16  2456016      10.0        1.0        0.0
4        17   245585       1.0        0.0        0.0
5        18   245624       1.0        0.0        0.0
6        20  1964545      22.0        9.0       10.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      80.0        0.0        0.0
1        12       2.0        0.0        0.0
2        15      31.0       18.0        0.0
3        16      10.0        1.0        0.0
4        17       1.0        0.0        0.0
5        18       1.0        0.0        0.0
6        20      22.0        9.0       10.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   80.0
1        12  Released    2.0
2        15  Released   31.0
3        16  Released   10.0
4        17  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15    16   17   18    20
Status                                          
Completed   0.0  0.0   0.0   0.0  0.0  0.0  10.0
Executing   0.0  0.0  18.0   1.0  0.0  0.0   9.0
Released   80.0  2.0  31.0  10.0  1.0  1.0  22.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15    16   17   18    20
0          Completed   0.0  0.0   0.0   0.0  0.0  0.0  10.0
1          Executing   0.0  0.0  18.0   1.0  0.0  0.0   9.0
2           Released  80.0  2.0  31.0  10.0  1.0  1.0  22.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15    16   17   18    20
0  Completed   0.0  0.0   0.0   0.0  0.0  0.0  10.0
1  Executing   0.0  0.0  18.0   1.0  0.0  0.0   9.0
2   Released  80.0  2.0  31.0  10.0  1.0  1.0  22.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()